### Prerequisites

To run this .NET Interactive run book, you need a create an AppRegistration in Azure AD with the following settings
- Authentication -> Platform: Mobile and Desktop
- Authentication -> Platform -> RedirectUris: `http://localhost`
- Authentication -> Supported Account Types: Multi tenant
- Press **Save**
- API Permissions -> Add a Permission -> Delegated Permisions -> MS Graph: `Application.ReadWrite.All`

We start by importing the necessary NuGet packages to be used throughout this runbook

In [ ]:
#r "nuget:Microsoft.Graph"
#r "nuget:Azure.Identity"

### Create the MSAL Client
Ensure that you add the appropriate `usings` so that we can interact with Graph and AAD.

Instantiate the MSAL Client to authenticate against AAD and get the right Graph Permissions.
The MSAL Client requires the 2 following properties:
- `ClientID` (From the Overview tab of your AAD App Registration)
- `TenantID` (From the Overview tab. Can be the tenant id [Guid] or the tenant name)

In [ ]:
using Azure.Identity;
using System.Threading.Tasks;

public static InteractiveBrowserCredential GetCredentials()
{
    var clientId = "f59bc665-04ce-4191-a3b2-99f65a1d740e";
    var tenantId = "common";
    
    var options = new InteractiveBrowserCredentialOptions
    {
        TenantId = tenantId,
        ClientId = clientId,
        AuthorityHost = AzureAuthorityHosts.AzurePublicCloud,
        RedirectUri = new Uri("http://localhost"),
    };

    return new InteractiveBrowserCredential(options);
}

### Create an API App Registration in Azure AD

Right now the scope is hardcoded to `access_as_user`. Feel free to change this to meet your needs

In [ ]:
using Microsoft.Graph;

var apiAppName = ".NET Conf Demo API";
var scopeName = "access_as_user";

var graphClient = new GraphServiceClient(GetCredentials(), new string[] {"Application.ReadWrite.All"});
var apiApp = GetApiApplication(apiAppName);

var apiAppRegistration = await graphClient.Applications
                            .Request()
                            .AddAsync(apiApp);
                            
UpdateAppAPISettings(ref apiAppRegistration);

await graphClient.Applications[apiAppRegistration.Id]
                    .Request()
                    .UpdateAsync(apiAppRegistration);  

var tenantId = await GetTenantId();

Console.WriteLine($"Client Id: {apiAppRegistration.AppId}");
Console.WriteLine($"Domain: {apiAppRegistration.PublisherDomain}");
Console.WriteLine($"Tenant Id: {tenantId}");
Console.WriteLine($"API URI: {apiAppRegistration.IdentifierUris.First()}/{scopeName}");

Application GetApiApplication(string displayName) => new Application
{
    DisplayName = displayName
};

async Task<string> GetTenantId()
{
    var organization = await graphClient.Organization
                                .Request()
                                .GetAsync();

    return organization.FirstOrDefault().Id;
}

void UpdateAppAPISettings(ref Application app)
{
    app.SignInAudience = "AzureADMyOrg";
    app.IdentifierUris = new List<String>()
    {
        $"api://{app.AppId}"
    };
    
    app.Api = new ApiApplication
    {
        AcceptMappedClaims = null,
        KnownClientApplications = new List<Guid>()
        {
        },
        RequestedAccessTokenVersion = 2,
        Oauth2PermissionScopes = new List<PermissionScope>()
        {
            new PermissionScope
            {
                Id = Guid.NewGuid(),
                AdminConsentDescription = "access the app as a user",
                AdminConsentDisplayName = "access the app as a user",
                IsEnabled = true,
                Type = "User",
                UserConsentDescription = "access the app as a user",
                UserConsentDisplayName = "access the app as a user",
                Value = scopeName
            }
        },
        PreAuthorizedApplications = new List<PreAuthorizedApplication>()
        {
        }
    };
}

### Create the Service Principal
Next we need to create a Service Principal that maps to the App Registration. 
This registers the app with the current Tenant and make the App Registration available to 
the tenant's apps and users to authenticate against and/or acquire tokens.

In [ ]:
using Microsoft.Graph;

var servicePrincipal = await graphClient.ServicePrincipals
                            .Request()
                            .AddAsync(CreateServicePrincipal(apiAppRegistration.AppId));

private static ServicePrincipal CreateServicePrincipal(string appId) => new ServicePrincipal
{
    AppId = appId
};

### Web App App Registration
Create a client App Registration for a server-side **web app** (.NET, Node, Java etc).

Users signing in to the client app, will be able to request an access token for the API ().
You'll need to set the following two properties before running this script:
- `clientAppName` -> the name for your App Registration
- `redirectUri` -> the URI where your app expects the returned tokens. For .NET web apps, 
it's usually `https://localhost:5001/signin-oidc`

In [ ]:
using Microsoft.Graph;

var clientAppName = "Thunder Client App";
var redirectUri = "https://www.thunderclient.io/oauth/callback";

var clientApp = GetClientApplication(
    apiAppRegistration,
    clientAppName,
    redirectUri);

var clientAppRegistration = await graphClient.Applications
                                .Request()
                                .AddAsync(clientApp);

Console.WriteLine($"Client Id: {clientAppRegistration.AppId}");
Console.WriteLine($"Auth Endpoint: https://login.microsoftonline.com/{tenantId}/oauth2/v2.0/authorize");
Console.WriteLine($"Token Endpoint https://login.microsoftonline.com/{tenantId}/oauth2/v2.0/token");

private static Application GetClientApplication(Application app,string displayName, string redirectUri) 
    => new Application
{
    DisplayName = displayName,
    Web = new WebApplication
    {
        RedirectUris = new List<string>(){redirectUri}
    },
    RequiredResourceAccess = new List<RequiredResourceAccess>()
    {
        new RequiredResourceAccess
        {
            ResourceAppId = app.AppId,
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {
                    Id = app.Api.Oauth2PermissionScopes.FirstOrDefault().Id,
                    Type = "Scope"
                }
            }
        },
        new RequiredResourceAccess
        {
            // OIDC - Graph scope (user.read)
            ResourceAppId = "00000003-0000-0000-c000-000000000000",
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {     
                    Id = new Guid("e1fe6dd8ba314d6189e788639da4683d"),
                    Type = "Scope"
                }
            }
        }
    } 
};

### Add a Client Secret

In [ ]:
using Microsoft.Graph;

var passwordCredential = new PasswordCredential
{
	DisplayName = "delete me"
};

var credential = await graphClient.Applications[$"{clientAppRegistration.Id}"]
	.AddPassword(passwordCredential)
	.Request()
	.PostAsync();

Console.WriteLine($"Client Secret: {credential.SecretText}");